# Clase Práctica de NLP: Soluciones

Este cuaderno contiene las soluciones completas para los ejercicios prácticos.

---

## Configuración Inicial

In [ ]:
!python -m spacy download es_core_news_sm
!python -m spacy download es_core_news_md

In [ ]:
import spacy
import es_core_news_sm
import es_core_news_md
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

## Ejercicio 1: Limpieza y Vectorización (Solución)

In [ ]:
# Datos de prueba
frases = [
    "La inteligencia artificial está transformando el mundo.",
    "El aprendizaje automático es una rama de la inteligencia artificial.",
    "Me gusta comer pizza con piña los fines de semana.",
    "Los sistemas de IA aprenden de los datos."
]

# 1. Cargar modelo
nlp = es_core_news_sm.load()

# 2. Definir función de preprocesamiento
def preprocesar(texto):
    doc = nlp(texto)
    # Filtrar stopwords y puntuación, y obtener lemas
    tokens_limpios = [
        token.lemma_.lower() 
        for token in doc 
        if not token.is_stop and token.pos_ != 'PUNCT'
    ]
    return " ".join(tokens_limpios)

# 3. Procesar las frases
frases_limpias = [preprocesar(f) for f in frases]
print("Frases limpias:", frases_limpias)

# 4. Vectorizar con TF-IDF
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(frases_limpias)

print("\nDimensiones de la matriz:", matriz_tfidf.shape)
print("Vocabulario:", vectorizer.get_feature_names_out())

## Ejercicio 2: Buscador de Similitud (Solución)

In [ ]:
query = "La IA cambia la sociedad"

# 1. Preprocesar query
query_limpia = preprocesar(query)
print(f"Query procesada: {query_limpia}")

# 2. Vectorizar query (usando transform, NO fit_transform)
query_vec = vectorizer.transform([query_limpia])

# 3. Calcular similitud
similitudes = cosine_similarity(query_vec, matriz_tfidf)

# 4. Mostrar resultados
print("\nResultados de similitud:")
for i, sim in enumerate(similitudes[0]):
    print(f"Frase: '{frases[i]}' | Similitud: {sim:.4f}")

idx_max = np.argmax(similitudes)
print(f"\nLa frase más similar es: '{frases[idx_max]}'")

### Reto Extra: LSA

In [ ]:
lsa = TruncatedSVD(n_components=2)
matriz_lsa = lsa.fit_transform(matriz_tfidf)
query_lsa = lsa.transform(query_vec)

similitudes_lsa = cosine_similarity(query_lsa, matriz_lsa)
print("Similitudes con LSA:", similitudes_lsa)

## Ejercicio 3: Explorando Significados con Embeddings (Solución)

In [ ]:
# 1. Cargar modelo con vectores
nlp_md = es_core_news_md.load()

palabras = ["perro", "gato", "coche", "moto", "manzana", "banana", "ordenador"]

# 2. Obtener vectores
vectores = [nlp_md(p).vector for p in palabras]
vectores = np.array(vectores)

# 3. Calcular similitudes (ejemplo)
token1 = nlp_md("perro")
token2 = nlp_md("gato")
print(f"Similitud perro-gato: {token1.similarity(token2):.4f}")

token3 = nlp_md("perro")
token4 = nlp_md("coche")
print(f"Similitud perro-coche: {token3.similarity(token4):.4f}")

# 4. Visualización con PCA
pca = PCA(n_components=2)
vectores_2d = pca.fit_transform(vectores)

plt.figure(figsize=(8,6))
plt.scatter(vectores_2d[:, 0], vectores_2d[:, 1], c='red')

for i, palabra in enumerate(palabras):
    plt.text(vectores_2d[i, 0]+0.02, vectores_2d[i, 1]+0.02, palabra, fontsize=12)

plt.title("Visualización de Embeddings con PCA")
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.grid(True)
plt.show()